In [ ]:
import pandas as pd 
import warnings 
import tkinter as tk

warnings.filterwarnings('ignore')

In [ ]:
cols = 'user_id item_id rating timespan'.split() #declaring column names for data frame which contains the rating that user has given to movie

df = pd.read_csv(r'E:\AI Training\movie re\u.data',sep='\t', names=cols) #creating DataFrame which contains ratings and unique item_id

cols_movies = ['item_id', 'title']+[str(i) for i in range(22)] #declaring column names for data frame which contains movie titles and unique user_id

movie_titles = pd.read_csv(r'E:\AI Training\movie re\u.item',sep='|', names = cols_movies ,encoding='latin-1') #creating DataFrame which contains movie name and unique item_id

movie_titles = movie_titles[['item_id','title']]

data = pd.merge(df,movie_titles, on='item_id') #merging above DataFrames on item_id as it is common to both 

rating = pd.DataFrame(data.groupby('title')['rating'].mean()) #calculating the mean of ratings so that a single rating of a movie can be obtained

rating['count'] = data['title'].value_counts() #counting how many times a movie has been rated 

pivot_df = data.pivot_table(index = "user_id", columns='title',values='rating') #making a pivot table so that we know which user has rated which movie 

#GUI

app = tk.Tk() #creating the instance of class Tk
app.title('Recommendor System') #assigning the title of our frame 
app.geometry('400x250') #making the dimensions of our frame

app.configure(bg="deep sky blue") #setting background color to deep sky blue

#defining lables
tk.Label(app,text='Movie you Watched', bg='#00BFFF').place(x=25,y=25)
tk.Label(app,text='Recommanded movies: ', bg='#00BFFF').place(x=5,y=100)
tk.Label(app,text='Also try: ', bg='#00BFFF').place(x=25,y=125)

#defining variable to store the value entered by user
movie_var = tk.Variable(app)
tk.Entry(app, textvariable=movie_var, width=28).place(x=150,y=25)

#defining labels which will we output or the recommendation
recommand_var = tk.Variable(app)
recommand_var.set("Try movie name...")
tk.Label(app,textvariable=recommand_var, bg='#00BFFF').place(x=150,y=100)

also_try_var = tk.Variable(app)
also_try_var.set("Try movie name...")
tk.Label(app,textvariable=also_try_var, bg='#00BFFF').place(x=150,y=125)


def find_recommendation():
    movie = movie_var.get().lower().strip() 
    try:
        #lambda x: movie in x.lower() will check if the movie name entered is in our dataset or not
        #if movie name entered by user is found or any sub string of user entered name is found then
        #movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0] will return the name of movie as in dataset
       
        movie = movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]   
        print(movie)
        movie_var.set(movie)
    
    #if movie is not found then except will execute
    except IndexError: 
        recommand_var.set('Movie not found')
        also_try_var.set('Movie not found')
    
    #if try is executed successfully then else will be executed
    else:
        print('Now we we\'ll find the suggestions')
        
        #making correlation dataframe 
        corr_df_1 = pd.DataFrame(pivot_df.corrwith(pivot_df[movie]), columns = ['Correlation'])
        
        #dropping rows with null data
        corr_df_1.dropna(inplace=True)
        
        corr_df_1 = corr_df_1.join(rating['count'])
        recommand_var.set(corr_df_1[(corr_df_1['count']>200)&(corr_df_1['Correlation']>0.4)].sort_values(by='Correlation', ascending=False).index[1])
        also_try_var.set(corr_df_1[(corr_df_1['Correlation']>0.4)].sort_values(by='Correlation', ascending=False).index[0]) 
            
#defining button to get the desires output
tk.Button(app, text='Find Suggestion', command=find_recommendation, bg='#000000',fg='#ffffff').place(x=125,y=60)

app.mainloop()